# Building Machine Learning Classifiers: Explore Random Forest model with grid-search

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

<>:22: SyntaxWarning: invalid escape sequence '\W'
<>:22: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Electrova\AppData\Local\Temp\ipykernel_21548\501319373.py:22: SyntaxWarning: invalid escape sequence '\W'
  tokens = re.split('\W+', text)
C:\Users\Electrova\AppData\Local\Temp\ipykernel_21548\501319373.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Building own Grid-search

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [4]:
def train_RF(n_est,depth):
    rf=RandomForestClassifier(n_estimators=n_est,max_depth=depth,n_jobs=-1)
    rf_model=rf.fit(X_train,y_train)
    y_pred=rf_model.predict(X_test)
    precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam',average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(n_est,depth,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))

In [6]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
for n_est in [10,50,100]:
    for depth in [10,20,30,None]:
        train_RF(n_est,depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.279 / Accuracy: 0.9
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.604 / Accuracy: 0.945
Est: 10 / Depth: 30 ---- Precision: 0.991 / Recall: 0.695 / Accuracy: 0.957
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.786 / Accuracy: 0.97
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.318 / Accuracy: 0.906
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.643 / Accuracy: 0.951
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.708 / Accuracy: 0.96
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.812 / Accuracy: 0.974
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.286 / Accuracy: 0.901
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.61 / Accuracy: 0.946
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.721 / Accuracy: 0.961
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.805 / Accuracy: 0.973
